In [16]:
import numpy as np
from biom.table import Table
from biom import load_table
import pandas as pd


In [19]:
t = load_table("/Volumes/TBHD/Valencia/Microbiome_Analysis/Nephele_cloud_play_project/outputs/for_analyze_with_microbiomedb.biom")
# print(t)
t.ids(axis='observation')

array(['TAX1', 'TAX4', 'TAX7', ..., 'TAX10235', 'TAX10238', 'TAX10239'],
      dtype='<U8')

In [51]:
phylum_idx = 6
collapse_f = lambda id_, md: '; '.join(md['Taxonomy'][6:phylum_idx + 1])
collapsed = t.collapse(collapse_f, axis='observation')

df = collapsed.to_dataframe()
display(df)

col1 = pd.DataFrame(df.loc[:, 'AAA00108'])
# display(col1)

col1 = col1.sparse.to_dense()

,AAA00108,AAA00405,AAA00507,AAA00607,AAA00806,AAA00910
Korarchaeum_cryptofilum,2.0,5.0,4.0,0.0,3.0,0.0
Prometheoarchaeum_syntrophicum,70.0,11.0,56.0,8.0,47.0,30.0
Mancarchaeum_acidiphilum,0.0,4.0,1.0,0.0,1.0,0.0
Micrarchaeum_sp.,0.0,2.0,3.0,0.0,1.0,0.0
Micrarchaeota_sp,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
Gordonia_virus_Yvonnetastic,0.0,0.0,1.0,0.0,1.0,0.0
Siphoviridae_sp,8.0,2.0,23.0,13.0,7.0,8.0
Caudovirales_sp,0.0,0.0,0.0,0.0,0.0,20.0
Wuhan_heteroptera_virus_3,0.0,0.0,0.0,0.0,0.0,1.0


In [58]:
pct = col1.div(col1.sum(axis=0), axis=1)
display(pct)
col1_largest = pct.nlargest(10, 'AAA00108')
display(col1_largest)
# col1_largest.plot.bar(stacked=True)

,AAA00108
Korarchaeum_cryptofilum,6.204035e-08
Prometheoarchaeum_syntrophicum,2.171412e-06
Mancarchaeum_acidiphilum,0.000000e+00
Micrarchaeum_sp.,0.000000e+00
Micrarchaeota_sp,3.102018e-08
...,...
Gordonia_virus_Yvonnetastic,0.000000e+00
Siphoviridae_sp,2.481614e-07
Caudovirales_sp,0.000000e+00
Wuhan_heteroptera_virus_3,0.000000e+00


,AAA00108
Unclassified_sp,0.617658
Anaerostipes_hadrus,0.058736
Ruminococcus_gnavus,0.049764
Anaerobutyricum_hallii,0.034341
Streptococcus_thermophilus,0.020750
Lachnospiraceae_sp,0.017942
Coprococcus_comes,0.014118
Ruminococcus_bicirculans,0.013722
Blautia_obeum,0.009922
Eubacteriales_sp,0.008663
